In [1]:
#dependencies and setup
import pandas as pd
import os
from openpyxl import Workbook
import numpy as np
import requests
import json
import re
from pprint import pprint
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

# Extraction

## School performance score 2020

In [7]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\mosab\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [8]:
# visit the NJ schools url
nj_school_rate_URL = 'https://patch.com/new-jersey/pointpleasant/every-nj-school-graded-2020-state-report-where-do-you-rank?fbclid=IwAR3F6wscWhMGHzRk2hy4MeZJ426hdUOFCF8qDjHzFiI7B3RfxshE9imZdyg'
browser.visit(nj_school_rate_URL)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [15]:
# access html elements to get the link for the table data
ol = soup.find('ol')
li = ol.find_all('li')
# iframe = div.find('iframe')
# table_data_link = iframe['src'].split("?")[0]
pprint(li)
# print(table_data_link)

[<li>Marine Academy of Science and Technology, Monmouth County Vocational School District, Monmouth 100.00</li>,
 <li>Mendham Township Middle School, Mendham Township School District, Morris 100.00</li>,
 <li>Central Jersey College Prep Charter School, Central Jersey College Prep Charter School, Charters 100.00</li>,
 <li>Markham Place School, Little Silver Boro School District, Monmouth 99.94</li>,
 <li>Jersey City Global Charter School, Jersey City Global Charter School, Charters 99.88</li>,
 <li>Clark Mills School, Manalapan-Englishtown Regional Schools School District, Monmouth 99.82</li>,
 <li>Riverview Elementary School, Denville Township K-8 School District, Morris 99.76</li>,
 <li>Academy For Information Technology, Union County Vocational-Technical School District, Union 99.72</li>,
 <li>Henry B. Milnes Elementary School, Fair Lawn Public School District, Bergen 99.70</li>,
 <li>Alexander Hamilton School, Glen Rock Public School District, Bergen 99.64</li>,
 <li>Hillside Eleme

In [18]:
records = []
for elem in li:
    records.append(elem.text.split(",",4))
print(records)

[['Marine Academy of Science and Technology', ' Monmouth County Vocational School District', ' Monmouth 100.00'], ['Mendham Township Middle School', ' Mendham Township School District', ' Morris 100.00'], ['Central Jersey College Prep Charter School', ' Central Jersey College Prep Charter School', ' Charters 100.00'], ['Markham Place School', ' Little Silver Boro School District', ' Monmouth 99.94'], ['Jersey City Global Charter School', ' Jersey City Global Charter School', ' Charters 99.88'], ['Clark Mills School', ' Manalapan-Englishtown Regional Schools School District', ' Monmouth 99.82'], ['Riverview Elementary School', ' Denville Township K-8 School District', ' Morris 99.76'], ['Academy For Information Technology', ' Union County Vocational-Technical School District', ' Union 99.72'], ['Henry B. Milnes Elementary School', ' Fair Lawn Public School District', ' Bergen 99.70'], ['Alexander Hamilton School', ' Glen Rock Public School District', ' Bergen 99.64'], ['Hillside Element

In [19]:
# close the browser session    
browser.quit()

In [ ]:
temp = re.findall(r'\d+', records[i][2]) 
res = list(map(int, temp)) 
re.sub(r'[^A-Za-z0-9 ]+', '', s)

In [59]:
county_name = []
district_name = []
school_name = []
rating = []
for i in range(len(records)):
    l =len(records[i])
    county_name.append(re.sub(r'[^A-Za-z ]+', '', records[i][l-1]).upper().strip())
    rating.append(re.sub(r'[^0-9.]','', records[i][l-1]))
    district_name.append(records[i][l-2].strip().upper())
    school_name.append(records[i][l-3].strip().upper())

In [63]:
school_ratings_2020_df = pd.DataFrame({"county_name": county_name,
                                    "district_name": district_name,
                                    "school_name": school_name,
                                    "rating": rating})
school_ratings_2020_df.sort_values(by =["county_name", "district_name", "school_name"], ascending=True, inplace=True, ignore_index=True)
school_ratings_2020_df.drop(school_ratings_2020_df[school_ratings_2020_df.county_name =='CHARTERS'].index, inplace=True)
school_ratings_2020_df.head()

,county_name,district_name,school_name,rating
0,ATLANTIC,ABSECON PUBLIC SCHOOLS DISTRICT,EMMA C ATTALES,16.30
1,ATLANTIC,ABSECON PUBLIC SCHOOLS DISTRICT,H ASHTON MARSH,34.09
2,ATLANTIC,ATLANTIC CITY SCHOOL DISTRICT,ATLANTIC CITY HIGH SCHOOL,11.27
3,ATLANTIC,ATLANTIC CITY SCHOOL DISTRICT,BRIGHTON AVENUE SCHOOL,21.30
4,ATLANTIC,ATLANTIC CITY SCHOOL DISTRICT,CHELSEA HEIGHTS SCHOOL,37.89


In [66]:
len(school_ratings_2020_df)

2008

## School performance score 2017-2018

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the Mars News url
nj_school_rate_URL = 'https://www.nj.com/education/2019/03/nj-just-graded-its-public-schools-from-0-100-heres-how-every-school-scored.html'
browser.visit(nj_school_rate_URL)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# access html elements to get the link for the table data
div = soup.find('div', class_ = 'rawhtml')
iframe = div.find('iframe')
table_data_link = iframe['src'].split("?")[0]
# pprint(iframe)
print(table_data_link)

In [ ]:
# go to the file page in S3
browser.visit(table_data_link)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
div = soup.find('div', class_ = 'download')
school_rating_file_url = div.find('a', href = True)['href']
print(school_rating_file_url)

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
# import the csv from the URL into dataframe 
dtype_dic= {'COUNTY_CODE': str, 
            'DISTRICT_CODE' : str,
           "SCHOOL_CODE": str}
school_rating_df = pd.read_csv(school_rating_file_url, dtype = dtype_dic)
# school_data.columns
school_rating_df = school_rating_df[['COUNTY_CODE', 'COUNTY_NAME', 'DISTRICT_CODE', 'DISTRICT_NAME',
       'SCHOOL_CODE', 'SCHOOL_NAME', 'GRADESPAN', 'PRINCIPAL_NAME', 'ADDRESS',
       'CITY_STATE_ZIP', 'PHONE', 'EMAIL', 'WEBSITE', 'FACEBOOK', 'TWITTER',
       'CutOffScoreTargetedSupImp', 'Targeted_LowPerf', 'ELAProf_MetTarget',
       'MathProf_MetTarget', 'Grad4YR_MetTarget', 'Grad5YR_MetTarget',
       'ELAGrowth_MetStandard', 'MathGrowth_MetStandard', 'ELP_MetTarget',
       'CA_MetAverage', 'Targeted_ConsUnderPerf', 'SummativeRating', 'SummativeScore']]
school_rating_df.head()

In [ ]:
# drop charter school records
school_rating_df.drop(school_rating_df[school_rating_df.COUNTY_NAME =='CHARTERS'].index, inplace=True)

In [ ]:
# unique counties
school_rating_df['COUNTY_CODE'].nunique()

In [ ]:
# make all the column name lower case
school_rating_df.columns= school_rating_df.columns.str.lower()
school_rating_df.head()

### Offline copy import

In [ ]:
# File to Load (Remember to Change These)
# source = https://www.nj.com/education/2019/03/nj-just-graded-its-public-schools-from-0-100-heres-how-every-school-scored.html
# file_to_load = "data/school_scores_2017_18.csv"
# dtype_dic= {'COUNTY_CODE': str, 
#             'DISTRICT_CODE' : str,
#            "SCHOOL_CODE": str}
# Read school score File and store into Pandas data frame
# school_data = pd.read_csv(file_to_load, dtype = dtype_dic)
# school_data = school_data[['COUNTY_CODE', 'COUNTY_NAME', 'DISTRICT_CODE', 'DISTRICT_NAME',
#        'SCHOOL_CODE', 'SCHOOL_NAME', 'GRADESPAN', 'PRINCIPAL_NAME', 'ADDRESS',
#        'CITY_STATE_ZIP', 'PHONE', 'EMAIL', 'WEBSITE', 'FACEBOOK', 'TWITTER',
#        'CutOffScoreTargetedSupImp', 'Targeted_LowPerf', 'ELAProf_MetTarget',
#        'MathProf_MetTarget', 'Grad4YR_MetTarget', 'Grad5YR_MetTarget',
#        'ELAGrowth_MetStandard', 'MathGrowth_MetStandard', 'ELP_MetTarget',
#        'CA_MetAverage', 'Targeted_ConsUnderPerf', 'SummativeRating', 'SummativeScore']]
# school_data.head()

## Poverty 2019

In [ ]:
# URL for county poverty data
poverty_url = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT,SAEMHI_MOE,SAEPOVALL_PT,SAEPOVALL_MOE,SAEPOVRTALL_MOE,SAEPOVRTALL_PT,STABREV&for=county:*&in=state:34&YEAR=2019'

In [ ]:
# API call to get countries metadata
poverty_data = requests.get(poverty_url).json()
pprint(poverty_data)

In [ ]:
# initialize empty lists
county_name = []
median_hh_income = []
median_hh_inc_moe = []
poverty_count = []
poverty_count_moe = []
poverty_rate = []
poverty_rate_moe = []
st_abb =[]
state_fips = []
county_fips = []

In [ ]:
# iterate through the response to populate the lists and create a dataframe
for i in range(1,len(poverty_data)):
    county_name.append(poverty_data[i][0])
    median_hh_income.append(poverty_data[i][1])
    median_hh_inc_moe.append(poverty_data[i][2])
    poverty_count.append(poverty_data[i][3])
    poverty_count_moe.append(poverty_data[i][4])
    poverty_rate.append(poverty_data[i][5])
    poverty_rate_moe.append(poverty_data[i][6])
    st_abb.append(poverty_data[i][7])
    state_fips.append(poverty_data[i][9])
    county_fips.append(poverty_data[i][10])
counties_poverty_df = pd.DataFrame({"county_name": county_name,
                                    "median_hh_income": median_hh_income,
                                    "median_hh_inc_moe": median_hh_inc_moe,
                                    "poverty_count": poverty_count,
                                    "poverty_count_moe": poverty_count_moe,
                                    "poverty_rate": poverty_rate,
                                    "poverty_rate_moe": poverty_rate_moe,
                                    "st_abb": st_abb,
                                    "state_fips": state_fips,
                                    "county_fips": county_fips})
counties_poverty_df.head()

In [ ]:
# Manipulating the column name to match across all the datasets
counties_poverty_df['county_name']= counties_poverty_df.county_name.str.replace("County","")
counties_poverty_df['county_name']= counties_poverty_df['county_name'].str.upper()
counties_poverty_df['county_name']= counties_poverty_df['county_name'].str.strip()
counties_poverty_df

In [ ]:
# unique county count
counties_poverty_df['county_name'].nunique()

## County crime reports 2020

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the Mars News url
crime_NJ_URL = 'https://www.njsp.org/ucr/current-crime-data.shtml'
browser.visit(crime_NJ_URL)
time.sleep(2)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# use splinter to click on the 'Continue' button to retrieve a full-size jpg url
browser.find_by_tag("input").last.click()
time.sleep(1)

In [ ]:
# create HTML object
html = browser.html
# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
# fetch the table
table = soup.find('table')
# fetch the partial link for donwloadable file
download_item = table.find_all('a', href = True)
download_link= download_item[0]['href']
# create full download link
base_url= 'https://www.njsp.org/ucr/'
full_file_link = base_url+download_link
print(full_file_link)

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
# create xlsx object with the file from the link
xls = pd.ExcelFile(full_file_link,
     engine='openpyxl')
# get all the worksheet names
crime_sheet = xls.sheet_names
print(crime_sheet)

In [ ]:
# unique county
len(crime_sheet[1:])

In [ ]:
# create separate dataframes for each worksheet
for i in range(1,len(crime_sheet)):
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")] = pd.read_excel(full_file_link,
     engine='openpyxl',sheet_name=crime_sheet[i], skiprows=range(3)).dropna(how="all")
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")]["county_name"] = '%s' % crime_sheet[i].upper()
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")].drop(globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")].tail(7).index, inplace = True)
    print('crime_%s' % crime_sheet[i].lower().replace(" ", "_"))

### Offline copy import

In [ ]:
# del [crime_atlantic,
# crime_bergen,
# crime_burlington,
# crime_camden,
# crime_cape_may,
# crime_cumberland,
# crime_essex,
# crime_gloucester,
# crime_hudson,
# crime_hunterdon,
# crime_mercer,
# crime_middlesex,
# crime_monmouth,
# crime_morris,
# crime_ocean,
# crime_passaic,
# crime_salem,
# crime_somerset,
# crime_sussex,
# crime_union,
# crime_warren]

In [ ]:
# source : https://www.njsp.org/ucr/current-crime-data1.shtml?agree=0
# xls = pd.ExcelFile(os.path.join("data", "2020_UCR_Jan-Sept_v2.xlsx"),
#      engine='openpyxl')

In [ ]:
# crime_sheet = xls.sheet_names
# crime_sheet

In [ ]:
# for i in range(1,len(crime_sheet)):
#     globals()['crime_%s' % crime_sheet[i].lower()] = pd.read_excel(os.path.join("data", "2020_UCR_Jan-Sept_v2.xlsx"),
#      engine='openpyxl',sheet_name=crime_sheet[i], skiprows=range(3)).dropna(how="all")
#     print('crime_%s' % crime_sheet[i].lower())

#### Atlantic crime

In [ ]:
crime_atlantic.head(-1)

In [ ]:
len(crime_atlantic)

In [ ]:
# concat all the dataframes
all_crime_df = pd.concat([crime_atlantic,
crime_bergen,
crime_burlington,
crime_camden,
crime_cape_may,
crime_cumberland,
crime_essex,
crime_gloucester,
crime_hudson,
crime_hunterdon,
crime_mercer,
crime_middlesex,
crime_monmouth,
crime_morris,
crime_ocean,
crime_passaic,
crime_salem,
crime_somerset,
crime_sussex,
crime_union,
crime_warren], ignore_index = True)

In [ ]:
# make all the column name lower case
all_crime_df.columns= all_crime_df.columns.str.lower()
all_crime_df.columns= all_crime_df.columns.str.replace(" ","_")
all_crime_df.head(10)

### Population dataset

In [ ]:
population_df = all_crime_df[["county_name","agency", "orinumber", "population"]].dropna(how="any")
population_df["population"] = population_df["population"].astype(int)
population_df.drop_duplicates(keep='first', inplace= True, ignore_index=True)
population_df.head(-10)

In [ ]:
# unique county count
population_df['county_name'].nunique()

In [ ]:
len(population_df)

### Crime detail

In [ ]:
pol_dep = []
for i in range(len(population_df["agency"])):
    for k in range(5):
        pol_dep.append(population_df["agency"][i])
print(pol_dep)

In [ ]:
len(pol_dep)

In [ ]:
crime_det_df = all_crime_df[["county_name","agency", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]].dropna(how="any")

crime_det_df.rename(columns = {"agency": "report_type"}, inplace =True)

crime_det_df = crime_det_df.loc[crime_det_df["report_type"] != crime_det_df["report_type"].shift()]
crime_det_df.reset_index(inplace= True, drop = True)
crime_det_df["police_dept"] = pol_dep
crime_det_df = crime_det_df[["county_name","police_dept", "report_type", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]]
crime_det_df.head(-1)

### Crime summary dataset

In [ ]:
crime_df = all_crime_df[["county_name","agency", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]].dropna(how="any")
crime_df.reset_index(inplace= True, drop = True)
crime_df.rename(columns = {"agency": "report_type"}, inplace =True)
crime_df = crime_df.groupby(['county_name', 'report_type'],as_index=False).agg(murder = ('murder', 'sum'), rape = ('rape', 'sum'),\
                                                               robbery = ('robbery', 'sum'),assault = ('assault', 'sum'),\
                                                               burglary = ('burglary', 'sum'),larceny = ('larceny', 'sum'),\
                                                               auto_theft = ('auto_theft', 'sum'),total = ('total', 'sum'))
crime_df.head(-10)

In [ ]:
# unique county count
crime_df['county_name'].nunique()

In [ ]:
# county record count
crime_df['county_name'].value_counts()

# DB load

In [ ]:
# Connect to the NJ_CPS database
connection  = sqlite3.connect("Resources/NJ_CPS.sqlite")

# Get a cursor object
cursor = connection.cursor()

# Execute the DROP Table SQL statement
drop_school = "DROP TABLE IF EXISTS NJ_school_rating"

cursor.execute(drop_school)

drop_poverty = "DROP TABLE IF EXISTS NJ_poverty"

cursor.execute(drop_poverty)

drop_population = "DROP TABLE IF EXISTS NJ_population"

cursor.execute(drop_population)

drop_crime = "DROP TABLE IF EXISTS NJ_crime"

cursor.execute(drop_crime)

drop_crime_det = "DROP TABLE IF EXISTS NJ_crime_detail"

cursor.execute(drop_crime_det)

In [3]:
#  SQLite DB creation and establishing connection
database_path = "Resources/NJ_CPS.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=True)
sqlite_connection = engine.connect()

2021-03-10 09:14:49,547 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-10 09:14:49,550 INFO sqlalchemy.engine.base.Engine ()
2021-03-10 09:14:49,552 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-10 09:14:49,552 INFO sqlalchemy.engine.base.Engine ()


## Loading school ratings in to DB

In [ ]:
sqlite_table = "NJ_school_rating"
school_rating_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading poverty rates in DB

In [ ]:
sqlite_table = "NJ_poverty"
counties_poverty_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading crime detail data in DB

In [ ]:
sqlite_table = "NJ_crime_detail"
crime_det_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading crime data in DB

In [ ]:
sqlite_table = "NJ_crime"
crime_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading population data in DB

In [ ]:
sqlite_table = "NJ_population"
population_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

# DB query Test

In [ ]:
metadata_df = pd.read_sql_query("SELECT T1.*, T2.population FROM (SELECT county_name,median_hh_income,poverty_rate FROM NJ_poverty) AS T1 \
                                INNER JOIN (SELECT county_name, sum(population) as population FROM NJ_population GROUP BY county_name) AS T2\
                                ON T1.county_name = T2.county_name", sqlite_connection)
metadata_df["median_hh_income"] = metadata_df["median_hh_income"].astype(int)
metadata_df

In [28]:
school_df = pd.read_sql_query("SELECT county_name,district_code||school_code as school_id, gradespan, school_name,summativescore FROM NJ_school_rating", sqlite_connection)
school_df.head()

2021-03-10 10:09:19,948 INFO sqlalchemy.engine.base.Engine SELECT county_name,district_code||school_code as school_id, gradespan, school_name,summativescore FROM NJ_school_rating
2021-03-10 10:09:19,950 INFO sqlalchemy.engine.base.Engine ()


,county_name,school_id,gradespan,school_name,summativescore
0,ATLANTIC,0010050,05-08,EMMA C ATTALES,43.95
1,ATLANTIC,0010060,PK-04,H ASHTON MARSH,55.49
2,ATLANTIC,0110010,09-12,ATLANTIC CITY HIGH SCHOOL,12.33
3,ATLANTIC,0110030,PK-08,SOVEREIGN AVENUE SCHOOL,47.31
4,ATLANTIC,0110050,PK-08,CHELSEA HEIGHTS SCHOOL,35.53


In [69]:
crime_df = pd.read_sql_query("SELECT county_name, murder,rape, robbery, assault, burglary, larceny, auto_theft, total FROM NJ_crime WHERE report_type = 'Number of Offenses'", sqlite_connection)
crime_df.head()

2021-03-10 11:34:52,884 INFO sqlalchemy.engine.base.Engine SELECT county_name, murder,rape, robbery, assault, burglary, larceny, auto_theft, total FROM NJ_crime WHERE report_type = 'Number of Offenses'
2021-03-10 11:34:52,886 INFO sqlalchemy.engine.base.Engine ()


,county_name,murder,rape,robbery,assault,burglary,larceny,auto_theft,total
0,ATLANTIC,12.0,45.0,130.0,331.0,598.0,2994.0,203.0,4313.0
1,BERGEN,5.0,45.0,107.0,249.0,547.0,4299.0,476.0,5728.0
2,BURLINGTON,6.0,59.0,55.0,287.0,470.0,2695.0,219.0,3791.0
3,CAMDEN,22.0,73.0,327.0,1008.0,871.0,5671.0,608.0,8580.0
4,CAPE MAY,0.0,13.0,20.0,84.0,194.0,1339.0,48.0,1698.0


In [ ]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate,
           CRM.murder, CRM.rape, CRM.robbery, CRM.assault, CRM.burglary, CRM.larceny, CRM.auto_theft, CRM.total,
           POP.population, SCH.school_rank
           FROM NJ_poverty AS POV 
           INNER JOIN NJ_crime AS CRM  ON POV.county_name = CRM.county_name AND report_type = 'Number of Offenses'
           INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY county_name) AS POP ON POV.county_name = POP.county_name
           INNER JOIN (SELECT county_name, AVG(summativescore) AS school_rank FROM NJ_school_rating GROUP BY county_name) AS SCH ON POV.county_name = SCH.county_name'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate, POV.county_fips,
               CRM.total AS total_offense, CRM2.total AS rate_per_100k, CRM3.total AS total_arrest,
               POP.population, SCH.school_rank
               FROM NJ_poverty AS POV 
               INNER JOIN NJ_crime AS CRM ON POV.county_name = CRM.county_name AND CRM.report_type = 'Number of Offenses'
               INNER JOIN NJ_crime AS CRM2 ON POV.county_name = CRM2.county_name AND CRM2.report_type = 'Rate Per 100,000'
               INNER JOIN NJ_crime AS CRM3 ON POV.county_name = CRM3.county_name AND CRM3.report_type = 'Number of Arrests'
               INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY county_name) AS POP ON POV.county_name = POP.county_name
               INNER JOIN (SELECT county_name, AVG(summativescore) AS school_rank FROM NJ_school_rating GROUP BY county_name) AS SCH ON POV.county_name = SCH.county_name'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT * from NJ_crime_detail'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT police_dept||"-"||report_type AS id ,report_type AS label, county_name||"-"||police_dept AS parent from NJ_crime_detail'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT county_name, 'murder' AS crime_type, murder AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'rape' AS crime_type, rape AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'robbery' AS crime_type, robbery AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'assault' AS crime_type, assault AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'burglary' AS crime_type, burglary AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'larceny' AS crime_type, larceny AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'auto_theft' AS crime_type, auto_theft AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        '''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query1 = '''SELECT DISTINCT "NJ-"||county_name AS id , county_name AS label,  '' AS parent, AVG(summativescore) AS value FROM 
        NJ_school_rating GROUP BY 1,2,3'''
df1 = pd.read_sql(query1, sqlite_connection)
df1.head()

In [ ]:
query2 = '''SELECT DISTINCT county_name||"-"||district_name AS id ,district_name AS label,  "NJ-"||county_name AS parent, AVG(summativescore) AS value FROM 
        NJ_school_rating GROUP BY 1,2,3'''
df2 = pd.read_sql(query2, sqlite_connection)
df2.head()

In [ ]:
query4 = '''SELECT gradespan, AVG(summativescore) AS value FROM NJ_school_rating GROUP BY 1'''
df4 = pd.read_sql(query4, sqlite_connection)
df4

In [ ]:
query3 = '''SELECT DISTINCT district_name||"-"||school_name AS id ,school_name AS label,  county_name||"-"||district_name AS parent, AVG(summativescore) AS value FROM 
    NJ_school_rating GROUP BY 1,2,3'''
df3 = pd.read_sql(query3, sqlite_connection)
df3.head()

In [88]:
query = '''SELECT county_name,district_name,gradespan,school_name,summativescore
            FROM 
              ( SELECT county_name,district_name,gradespan,school_name, summativescore,
                       ROW_NUMBER() OVER (PARTITION BY county_name
                                          ORDER BY summativescore DESC) AS rn
                FROM NJ_school_rating) AS tmp 
            WHERE rn <= 3
            ORDER BY county_name ; '''
test = pd.read_sql_query(query, sqlite_connection)
test

2021-03-10 13:41:21,464 INFO sqlalchemy.engine.base.Engine SELECT county_name,district_name,gradespan,school_name,summativescore
            FROM 
              ( SELECT county_name,district_name,gradespan,school_name, summativescore,
                       ROW_NUMBER() OVER (PARTITION BY county_name
                                          ORDER BY summativescore DESC) AS rn
                FROM NJ_school_rating) AS tmp 
            WHERE rn <= 3
            ORDER BY county_name ; 
2021-03-10 13:41:21,466 INFO sqlalchemy.engine.base.Engine ()


,county_name,district_name,gradespan,school_name,summativescore
0,ATLANTIC,MARGATE CITY,05-08,EUGENE A. TIGHE MIDDLE SCHOOL,81.30
1,ATLANTIC,ATLANTIC CO VOCATIONAL,09-12,ATLANTIC COUNTY INSTITUTE OF TECHNOLOGY,73.40
2,ATLANTIC,LINWOOD CITY,05-08,BELHAVEN MIDDLE SCHOOL,72.37
3,BERGEN,CRESSKILL BORO,PK-05,EDWARD H. BRYAN,97.47
4,BERGEN,FORT LEE BORO,KG-06,SCHOOL NO. 1,96.24
...,...,...,...,...,...
58,UNION,UNION COUNTY VOCATIONAL,09-12,ACADEMY FOR INFORMATION TECHNOLOGY,95.84
59,UNION,WESTFIELD TOWN,01-05,WASHINGTON ELEMENTARY SCHOOL,95.27
60,WARREN,GREAT MEADOWS REGIONAL,03-05,LIBERTY ELEMENTARY SCHOOL,82.45
61,WARREN,GREENWICH TWP,06-08,STEWARTSVILLE MIDDLE SCHOOL,79.41


In [98]:
query = '''SELECT county_name,district_name,gradespan,school_name,summativescore FROM NJ_school_rating ORDER BY county_name,district_name,gradespan,school_name;'''
test = pd.read_sql_query(query, sqlite_connection)
test

2021-03-10 15:04:39,027 INFO sqlalchemy.engine.base.Engine SELECT county_name,district_name,gradespan,school_name,summativescore FROM NJ_school_rating ORDER BY county_name,district_name,gradespan,school_name;
2021-03-10 15:04:39,029 INFO sqlalchemy.engine.base.Engine ()


,county_name,district_name,gradespan,school_name,summativescore
0,ATLANTIC,ABSECON CITY,05-08,EMMA C ATTALES,43.95
1,ATLANTIC,ABSECON CITY,PK-04,H ASHTON MARSH,55.49
2,ATLANTIC,ATLANTIC CITY,09-12,ATLANTIC CITY HIGH SCHOOL,12.33
3,ATLANTIC,ATLANTIC CITY,KG-08,TEXAS AVENUE SCHOOL,31.79
4,ATLANTIC,ATLANTIC CITY,PK-05,BRIGHTON AVENUE SCHOOL,37.82
...,...,...,...,...,...
2007,WARREN,WARREN HILLS REGIONAL,07-08,WARREN HILLS REGIONAL MIDDLE SCHOOL,22.68
2008,WARREN,WARREN HILLS REGIONAL,09-12,WARREN HILLS REGIONAL HIGH SCHOOL,50.88
2009,WARREN,WASHINGTON BORO,03-06,WASHINGTON MEMORIAL ELEMENTARY SCHOOL,62.59
2010,WARREN,WASHINGTON TWP,PK-06,BRASS CASTLE SCHOOL,55.43


In [92]:
flare = {}
flare["name"] = "flare"
flare["description"] = "flare"
flare

{'name': 'flare', 'description': 'flare'}

In [97]:
counties = list(test['county_name'].unique())
# counties

In [114]:
for index,row in test.loc[(test['county_name']=="ATLANTIC") & (test['district_name'] == "ATLANTIC CITY")].iterrows():
    print(row["school_name"],row["school_name"] row["summativescore"])

ATLANTIC CITY HIGH SCHOOL 12.33
TEXAS AVENUE SCHOOL 31.79
BRIGHTON AVENUE SCHOOL 37.82
CHELSEA HEIGHTS SCHOOL 35.53
DR MARTIN LUTHER KING JR SCHOOL COMPLEX 21.49
NEW YORK AVENUE SCHOOL 10.96
PENNSYLVANIA AVE SCHOOL 10.17
RICHMOND AVENUE SCHOOL 49.3
SOVEREIGN AVENUE SCHOOL 47.31
UPTOWN SCHOOL COMPLEX 7.82


In [113]:
for index,row in test.loc[(test['county_name']=="ATLANTIC") & (test['district_name'] == "ATLANTIC CITY") & (test['district_name'] == "PK-05")].iterrows():
    print(row["school_name"], row["summativescore"])

In [121]:
children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = counties[i]
    district = list(test['district_name'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(district)):
        child2 = {}
        child2["name"] = district[k]
        child2["description"] = district[k]
        child3_list = []
        gradespan = list(test['gradespan'].loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k])].unique())
        for j in range(len(gradespan)):
            child3 = {}
            child3["name"] = gradespan[j]
            child3["description"] = gradespan[j]
            child4_list = []
            for index,row in test.loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k]) & (test['gradespan'] == gradespan[j])].iterrows():
                child4 = {}
                child4["name"] = row["school_name"]
                child4["description"] = row["school_name"]
                child4["size"] = row["summativescore"]
                child4_list.append(child4)
            child3["children"] = child4_list
            child3_list.append(child3)
        child2["children"] = child3_list
        child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
children

[{'name': 'ATLANTIC',
  'description': 'ATLANTIC',
  'children': [{'name': 'ABSECON CITY',
    'description': 'ABSECON CITY',
    'children': [{'name': '05-08',
      'description': '05-08',
      'children': [{'name': 'EMMA C ATTALES',
        'description': 'EMMA C ATTALES',
        'size': 43.95}]},
     {'name': 'PK-04',
      'description': 'PK-04',
      'children': [{'name': 'H ASHTON MARSH',
        'description': 'H ASHTON MARSH',
        'size': 55.49}]}]},
   {'name': 'ATLANTIC CITY',
    'description': 'ATLANTIC CITY',
    'children': [{'name': '09-12',
      'description': '09-12',
      'children': [{'name': 'ATLANTIC CITY HIGH SCHOOL',
        'description': 'ATLANTIC CITY HIGH SCHOOL',
        'size': 12.33}]},
     {'name': 'KG-08',
      'description': 'KG-08',
      'children': [{'name': 'TEXAS AVENUE SCHOOL',
        'description': 'TEXAS AVENUE SCHOOL',
        'size': 31.79}]},
     {'name': 'PK-05',
      'description': 'PK-05',
      'children': [{'name': 'BRIGH

In [122]:
flare["children"] = children
flare

{'name': 'flare',
 'description': 'flare',
 'children': [{'name': 'ATLANTIC',
   'description': 'ATLANTIC',
   'children': [{'name': 'ABSECON CITY',
     'description': 'ABSECON CITY',
     'children': [{'name': '05-08',
       'description': '05-08',
       'children': [{'name': 'EMMA C ATTALES',
         'description': 'EMMA C ATTALES',
         'size': 43.95}]},
      {'name': 'PK-04',
       'description': 'PK-04',
       'children': [{'name': 'H ASHTON MARSH',
         'description': 'H ASHTON MARSH',
         'size': 55.49}]}]},
    {'name': 'ATLANTIC CITY',
     'description': 'ATLANTIC CITY',
     'children': [{'name': '09-12',
       'description': '09-12',
       'children': [{'name': 'ATLANTIC CITY HIGH SCHOOL',
         'description': 'ATLANTIC CITY HIGH SCHOOL',
         'size': 12.33}]},
      {'name': 'KG-08',
       'description': 'KG-08',
       'children': [{'name': 'TEXAS AVENUE SCHOOL',
         'description': 'TEXAS AVENUE SCHOOL',
         'size': 31.79}]},
      

In [123]:
with open("data/school.json", "w") as outfile:  
    json.dump(flare, outfile) 

In [ ]:
sqlite_connection.close()

# JSON creation

In [ ]:
crime_dict = crime_df.to_dict(orient='records')
pprint(crime_dict)

In [ ]:
metadata_dict = metadata_df.to_dict(orient='records')
pprint(metadata_dict)

In [76]:
import json

In [75]:
school_dict = school_df.to_dict(orient='records')
pprint(school_dict)

[{'county_name': 'ATLANTIC',
  'gradespan': '05-08',
  'school_id': '0010050',
  'school_name': 'EMMA C ATTALES',
  'summativescore': 43.95},
 {'county_name': 'ATLANTIC',
  'gradespan': 'PK-04',
  'school_id': '0010060',
  'school_name': 'H ASHTON MARSH',
  'summativescore': 55.49},
 {'county_name': 'ATLANTIC',
  'gradespan': '09-12',
  'school_id': '0110010',
  'school_name': 'ATLANTIC CITY HIGH SCHOOL',
  'summativescore': 12.33},
 {'county_name': 'ATLANTIC',
  'gradespan': 'PK-08',
  'school_id': '0110030',
  'school_name': 'SOVEREIGN AVENUE SCHOOL',
  'summativescore': 47.31},
 {'county_name': 'ATLANTIC',
  'gradespan': 'PK-08',
  'school_id': '0110050',
  'school_name': 'CHELSEA HEIGHTS SCHOOL',
  'summativescore': 35.53},
 {'county_name': 'ATLANTIC',
  'gradespan': 'KG-08',
  'school_id': '0110060',
  'school_name': 'TEXAS AVENUE SCHOOL',
  'summativescore': 31.79},
 {'county_name': 'ATLANTIC',
  'gradespan': 'PK-08',
  'school_id': '0110070',
  'school_name': 'NEW YORK AVENUE SC

 {'county_name': 'BERGEN',
  'gradespan': '09-12',
  'school_id': '4390050',
  'school_name': 'RIDGEWOOD HIGH SCHOOL',
  'summativescore': 85.93},
 {'county_name': 'BERGEN',
  'gradespan': '06-08',
  'school_id': '4390060',
  'school_name': 'BENJAMIN FRANKLIN MIDDLE SCHOOL',
  'summativescore': 54.64},
 {'county_name': 'BERGEN',
  'gradespan': '06-08',
  'school_id': '4390070',
  'school_name': 'GEORGE WASHINGTON MIDDLE SCHOOL',
  'summativescore': 45.66},
 {'county_name': 'BERGEN',
  'gradespan': 'KG-05',
  'school_id': '4390085',
  'school_name': 'HAWES ELEMENTARY SCHOOL',
  'summativescore': 80.39},
 {'county_name': 'BERGEN',
  'gradespan': 'KG-05',
  'school_id': '4390090',
  'school_name': 'RIDGE ELEMENTARY SCHOOL',
  'summativescore': 75.07},
 {'county_name': 'BERGEN',
  'gradespan': 'KG-05',
  'school_id': '4390100',
  'school_name': 'SOMERVILLE ELEMENTARY SCHOOL',
  'summativescore': 71.57},
 {'county_name': 'BERGEN',
  'gradespan': 'KG-05',
  'school_id': '4390110',
  'school_

  'summativescore': 1.18},
 {'county_name': 'CAMDEN',
  'gradespan': 'PK-08',
  'school_id': '0680180',
  'school_name': 'DR. HENRY H. DAVIS ELEMENTARY SCHOOL',
  'summativescore': 14.09},
 {'county_name': 'CAMDEN',
  'gradespan': 'PK-08',
  'school_id': '0680190',
  'school_name': 'THOMAS H. DUDLEY FAMILY SCHOOL',
  'summativescore': 5.55},
 {'county_name': 'CAMDEN',
  'gradespan': 'KG-05',
  'school_id': '0680205',
  'school_name': 'FOREST HILL SCHOOL',
  'summativescore': 1.74},
 {'county_name': 'CAMDEN',
  'gradespan': '06-12',
  'school_id': '0680245',
  'school_name': 'CREATIVE ARTS MORGAN VILLAGE ACADEMY',
  'summativescore': 45.56},
 {'county_name': 'CAMDEN',
  'gradespan': 'PK-08',
  'school_id': '0680300',
  'school_name': 'HARRY C. SHARP ELEMENTARY SCHOOL',
  'summativescore': 7.73},
 {'county_name': 'CAMDEN',
  'gradespan': '06-12',
  'school_id': '0680305',
  'school_name': 'CAMDEN BIG PICTURE LEARNING ACADEMY',
  'summativescore': 23.0},
 {'county_name': 'CAMDEN',
  'grad

 {'county_name': 'ESSEX',
  'gradespan': 'PK-05',
  'school_id': '2330120',
  'school_name': 'MADISON AT CHANCELLOR SOUTH',
  'summativescore': 27.87},
 {'county_name': 'ESSEX',
  'gradespan': 'PK-05',
  'school_id': '2330125',
  'school_name': 'THURGOOD MARSHALL ELEMENTARY SCHOOL',
  'summativescore': 48.53},
 {'county_name': 'ESSEX',
  'gradespan': 'PK-05',
  'school_id': '2330131',
  'school_name': 'MT. VERNON AVENUE ELEMENTARY SCHOOL',
  'summativescore': 33.04},
 {'county_name': 'ESSEX',
  'gradespan': '06-08',
  'school_id': '2330135',
  'school_name': 'UNIVERSITY MIDDLE SCHOOL',
  'summativescore': 19.52},
 {'county_name': 'ESSEX',
  'gradespan': 'KG-05',
  'school_id': '2330136',
  'school_name': 'UNIVERSITY ELEMENTARY SCHOOL',
  'summativescore': 12.8},
 {'county_name': 'ESSEX',
  'gradespan': '06-08',
  'school_id': '2330140',
  'school_name': 'UNION AVENUE MIDDLE SCHOOL',
  'summativescore': 34.11},
 {'county_name': 'ESSEX',
  'gradespan': '09-12',
  'school_id': '2730050',


 {'county_name': 'HUDSON',
  'gradespan': '06-08',
  'school_id': '2390095',
  'school_name': 'ACADEMY I',
  'summativescore': 76.84},
 {'county_name': 'HUDSON',
  'gradespan': 'PK-05',
  'school_id': '2390100',
  'school_name': 'FRANK R CONWELL SCHOOL',
  'summativescore': 58.44},
 {'county_name': 'HUDSON',
  'gradespan': '06-08',
  'school_id': '2390105',
  'school_name': 'MIDDLE SCHOOL # 4',
  'summativescore': 48.93},
 {'county_name': 'HUDSON',
  'gradespan': 'PK-08',
  'school_id': '2390110',
  'school_name': 'DR. MICHAEL CONTI SCHOOL',
  'summativescore': 56.53},
 {'county_name': 'HUDSON',
  'gradespan': 'PK-05',
  'school_id': '2390120',
  'school_name': 'CHARLES E. TREFURT SCHOOL',
  'summativescore': 49.08},
 {'county_name': 'HUDSON',
  'gradespan': 'PK-08',
  'school_id': '2390140',
  'school_name': 'MARTIN LUTHER KING, JR. SCHOOL',
  'summativescore': 63.18},
 {'county_name': 'HUDSON',
  'gradespan': 'PK-08',
  'school_id': '2390150',
  'school_name': 'JULIA A. BARNES SCHOOL

  'school_id': '2370080',
  'school_name': 'JOHN F. KENNEDY ELEMENTARY SCHOOL',
  'summativescore': 61.2},
 {'county_name': 'MIDDLESEX',
  'gradespan': '09-12',
  'school_id': '3120050',
  'school_name': 'METUCHEN HIGH SCHOOL',
  'summativescore': 71.44},
 {'county_name': 'MIDDLESEX',
  'gradespan': '01-04',
  'school_id': '3120060',
  'school_name': 'CAMPBELL ELEMENTARY SCHOOL',
  'summativescore': 68.0},
 {'county_name': 'MIDDLESEX',
  'gradespan': '05-08',
  'school_id': '3120070',
  'school_name': 'EDGAR MIDDLE SCHOOL',
  'summativescore': 71.24},
 {'county_name': 'MIDDLESEX',
  'gradespan': '09-12',
  'school_id': '3140050',
  'school_name': 'MIDDLESEX HIGH SCHOOL',
  'summativescore': 56.32},
 {'county_name': 'MIDDLESEX',
  'gradespan': '04-08',
  'school_id': '3140085',
  'school_name': 'VON E. MAUGER MIDDLE SCHOOL',
  'summativescore': 43.03},
 {'county_name': 'MIDDLESEX',
  'gradespan': '09-12',
  'school_id': '3150010',
  'school_name': 'MIDDLESEX COUNTY VOC ACADEMY MATH SCIE

 {'county_name': 'MONMOUTH',
  'gradespan': '05-08',
  'school_id': '3810040',
  'school_name': 'TOWNSHIP OF OCEAN INTERMEDIATE SCHOOL',
  'summativescore': 43.15},
 {'county_name': 'MONMOUTH',
  'gradespan': 'PK-04',
  'school_id': '3810060',
  'school_name': 'OCEAN TOWNSHIP ELEMENTARY SCHOOL',
  'summativescore': 51.52},
 {'county_name': 'MONMOUTH',
  'gradespan': 'PK-04',
  'school_id': '3810070',
  'school_name': 'WANAMASSA ELEMENTARY SCHOOL',
  'summativescore': 64.87},
 {'county_name': 'MONMOUTH',
  'gradespan': 'PK-04',
  'school_id': '3810080',
  'school_name': 'WAYSIDE ELEMENTARY SCHOOL',
  'summativescore': 62.07},
 {'county_name': 'MONMOUTH',
  'gradespan': '05-08',
  'school_id': '3830030',
  'school_name': 'MAPLE PLACE MIDDLE SCHOOL',
  'summativescore': 64.96},
 {'county_name': 'MONMOUTH',
  'gradespan': 'PK-04',
  'school_id': '3830050',
  'school_name': 'WOLF HILL ELEMENTARY SCHOOL',
  'summativescore': 84.88},
 {'county_name': 'MONMOUTH',
  'gradespan': '04-08',
  'sch

 {'county_name': 'OCEAN',
  'gradespan': '03-04',
  'school_id': '5020060',
  'school_name': 'MCKINLEY AVENUE ELEMENTARY SCHOOL',
  'summativescore': 39.79},
 {'county_name': 'OCEAN',
  'gradespan': '05-06',
  'school_id': '5020070',
  'school_name': 'STAFFORD INTERMEDIATE SCHOOL',
  'summativescore': 38.74},
 {'county_name': 'OCEAN',
  'gradespan': '09-12',
  'school_id': '5190030',
  'school_name': 'TOMS RIVER HIGH SCHOOL EAST',
  'summativescore': 29.38},
 {'county_name': 'OCEAN',
  'gradespan': '09-12',
  'school_id': '5190040',
  'school_name': 'TOMS RIVER HIGH SCHOOL NORTH',
  'summativescore': 44.02},
 {'county_name': 'OCEAN',
  'gradespan': '09-12',
  'school_id': '5190050',
  'school_name': 'TOMS RIVER HIGH SCHOOL SOUTH',
  'summativescore': 32.07},
 {'county_name': 'OCEAN',
  'gradespan': '06-08',
  'school_id': '5190060',
  'school_name': 'TOMS RIVER INTERMEDIATE SCHOOL EAST',
  'summativescore': 24.12},
 {'county_name': 'OCEAN',
  'gradespan': '06-08',
  'school_id': '51900

 {'county_name': 'SUSSEX',
  'gradespan': 'KG-06',
  'school_id': '1980050',
  'school_name': 'MARIAN E. MCKEOWN ELEMENTARY SCHOOL',
  'summativescore': 43.82},
 {'county_name': 'SUSSEX',
  'gradespan': '05-08',
  'school_id': '2030030',
  'school_name': 'HARDYSTON TOWNSHIP MIDDLE SCHOOL',
  'summativescore': 57.21},
 {'county_name': 'SUSSEX',
  'gradespan': 'PK-04',
  'school_id': '2030050',
  'school_name': 'HARDYSTON TOWNSHIP ELEMENTARY SCHOOL',
  'summativescore': 33.38},
 {'county_name': 'SUSSEX',
  'gradespan': '09-12',
  'school_id': '2165030',
  'school_name': 'HIGH POINT REGIONAL HIGH SCHOOL',
  'summativescore': 48.26},
 {'county_name': 'SUSSEX',
  'gradespan': '09-12',
  'school_id': '2240030',
  'school_name': 'HOPATCONG HIGH SCHOOL',
  'summativescore': 38.54},
 {'county_name': 'SUSSEX',
  'gradespan': '04-05',
  'school_id': '2240035',
  'school_name': 'DURBAN AVENUE ELEMENTARY SCHOOL',
  'summativescore': 35.01},
 {'county_name': 'SUSSEX',
  'gradespan': '06-08',
  'scho

In [ ]:
data_json= {}
data_json["metadata"] = metadata_dict
data_json["school"] = school_dict
data_json["crime"] = crime_dict

# Linear regression

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import scipy.stats as st
from scipy.stats import linregress
from scipy.stats import sem

In [ ]:
test.columns

In [ ]:
test.dtypes

In [ ]:
# Calculate the correlation coefficient and linear regression model 
x = test['median_hh_income'].astype(float)
# x = test['school_rank']
y = test['total_offense']

correlation = st.pearsonr(x,y)

# Print the answer to above calculation
print(f"""The correlation between school_rank and total_offense is {round(correlation[0],2)}.""")

# Calculate linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(20,36),fontsize=16,color="red")

# Add labels and title to plot
plt.xlabel("median_hh_income")
plt.ylabel("total_offense")
plt.title('total_offense by median_hh_income')
# plt.savefig("Images/scatter_line_plot.png")
plt.show()